In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [11]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json
import pprint

In [12]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v3.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [13]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
#     solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#     ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    ac = 100*sum(list(map(lambda x: x in prediction, solution)))/len(solution)
    return ac

In [33]:
np.set_printoptions(precision=6)
d = pickle.load(open('.temp/midScore.p','rb'), encoding='utf8')
f = d[0]
print(f.keys())
print()
print(f['sentences'])
print(f['0'].keys())
print(f['initialQuery'])
print(f['0']['updated_query'])
print(f['0']['final_score'])
print(np.asarray(f['0']['final_score'].argsort()))
print()
print(f['6']['updated_query'])
print(f['6']['final_score'])

dict_keys(['5', 'TransitionMat_t2t', 'initialQuery', 'prediction', '3', 'TransitionMat_w2w_samecng', '4', 'DCSLemmas', 'accuracy', 'TransitionMat_w2w', '2', 'nodeList', '1', '0', 'sentence', '6'])

dict_keys(['updated_query', 'final_score', 'removed', 'final_rank', 'w2w_samecng_rank', 'w2w_samecng_score', 'w2w_rank', 't2t_score', 't2t_rank', 'w2w_score', 'winner'])
[15, 16]
[15, 16, 21]
[[ 0.040709  0.04046   0.041591  0.041591  0.037939  0.041064  0.041101
   0.041317  0.041317  0.041353  0.041227  0.041433  0.041429  0.041316
   0.040558  0.031458  0.03466   0.041182  0.041182  0.041182  0.040675
   0.037607]]
[[15 16 21  4  1 14 20  0  5  6 17 18 19 10 13  8  7  9 12 11  2  3]]

[15, 16, 21, 4, 1, 20, 14, 10, 2]
[[ 0.040852  0.038907  0.040852  0.040852  0.038907  0.040852  0.040852
   0.040852  0.040852  0.040852  0.038907  0.040852  0.040852  0.040852
   0.038907  0.038072  0.038907  0.040852  0.040852  0.040852  0.038907
   0.038907]]


In [19]:
np.set_printoptions(precision=4)
for f in list(goodFileDict.keys())[1:10]:
# for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs[0:1]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj, verbose = False)
        if(result != None):            
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
            ac = Accuracy(result, dcsObj)
            print(f, "\t", ac)

115527.p 	 100.0
115597.p 	 80.0
411941.p 	 75.0
124269.p 	 66.66666666666667
70799.p 	 66.66666666666667
441765.p 	 42.857142857142854
143751.p 	 100.0


In [131]:
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)

0 [0]
1 [0]
Chosen: 1


UnboundLocalError: local variable 'stepDetails' referenced before assignment

In [7]:
fullfs = ['105587.p','293473.p','379245.p']

In [13]:
lowScore = []
midScore = []
highScore = []
allOfEm = []
np.set_printoptions(precision=4)
# for i in range(1, 1000):
#     f = list(goodFileDict.keys())[i]
# for f in list(goodFileDict.keys())[90:92]:
for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
#         print(f, i)
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        
        if(result != None):
#             print(runDetails['nodeList'])
            ac = runDetails['accuracy']
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]

            print(f, '\t', ac)
#             if(ac <= 100):
#                 print(result)
#                 print('vs')
#                 print(solution)
#                 pprint.pprint(runDetails['nodeList'])
#                 break
            # CODE FOR PRINTING RUNDETAILS
#             pprint.pprint(runDetails['nodeList'])
#             for i in range(runDetails['steps']):
#                 if str(i) in runDetails:
#                     print('Step', i)
#                     step = runDetails[str(i)]
#                     pprint.pprint(step['winner'])

            # CODE FOR SELECTING FILES 
#             if(i%100 == 0):
#                 print('Chekpoint:', i)
#             allOfEm.append((f, ac))
#             if(ac <= 40):
#                 lowScore.append((f, ac))
#             elif (ac <= 60):
#                 if(ac >= 50):
#                     midScore.append((f, ac))
#             elif (ac <= 100):
#                 if(ac >= 80):
#                     highScore.append((f, ac))

# print(len(allOfEm))
# print(len(lowScore))
# # print(lowScore)
# print(len(midScore))
# # print(midScore)
# print(len(highScore))
# # print(highScore)


105587.p 	 50.0
293473.p 	 66.66666666666667
379245.p 	 100.0


In [7]:
SeeSentence(sentenceObj)

SKT ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
Analyzing  upacAreRa
0 :  upacAreRa ['upacAra'] [{'noun': ['i. sg. m.']}]
Analyzing  SIlena
0 :  SIlena ['SIla'] [{'noun': ['i. sg. m.', 'i. sg. n.']}]
0 :  SIla ['SIla'] [{'compound': ['iic.']}]
0 :  SIle ['SIla'] [{'noun': ['loc. sg. m.', 'acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
0 :  SIla ['SIl'] [{'verb': ['imp. [1] ac. sg. 2']}]
3 :  ina ['na'] [{'noun': ['voc. sg. m.', 'voc. sg. n.']}]
4 :  na ['na'] [{'indeclinable': ['part.']}]
Analyzing  rUpayOvanasampadA
0 :  rUpa ['rUpa'] [{'compound': ['iic.']}]
10 :  sampadA ['sampad_2'] [{'noun': ['i. sg. f.']}]
4 :  yOvana ['yOvana'] [{'compound': ['iic.']}]
4 :  yO ['yu_3'] [{'noun': ['loc. sg. m.', 'loc. sg. f.']}]
6 :  vana ['vana'] [{'compound': ['iic.']}]
6 :  vana ['van'] [{'verb': ['imp. [1] ac. sg. 2']}]



In [8]:
SeeDCS(dcsObj)

DCS ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
[['upacāra'], ['śīla'], ['rūpa', 'yauvana', 'sampad']]
Lemmas: ['upacAra', 'SIla', 'rUpa', 'yOvana', 'sampad']
[['89'], ['91'], ['3', '3', '90']]



In [7]:
# lowScore = lowScore[0:10]
# midScore = midScore[0:10]
# highScore = highScore[0:10]
# print(len(lowScore))
# print(lowScore)
# print(len(midScore))
# print(midScore)
# print(len(highScore))
# print(highScore)

In [121]:
np.set_printoptions(precision=4)
# Clean the csv files
metPerfFH = open('.temp/metPerfFH.csv', 'w')
metPerfFH_bin = open('.temp/metPerfFH_bin.csv', 'w')

metPerfFH.write('w2w,t2t,w2w_samecng,InDCS\n')
metPerfFH_bin.write('w2w,t2t,w2w_samecng,InDCS\n')

metPerfFH.close()
metPerfFH_bin.close()
######################
gfl = list(goodFileDict.keys())
for i in range(0, 11000):
    f = gfl[i]
    if(i%250 == 0):
        print('Checkpoint:', i)
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        
        if(result != None):
            solution = [rom_slp(l) for ls in dcsObj.lemmas for l in ls]
#             ac = runDetails['accuracy']
#             print('Prediction:',result)
#             print('DCS:',solution)
#             print(f, '\t', ac)




Checkpoint: 0
Checkpoint: 250
Checkpoint: 500
Checkpoint: 750
Checkpoint: 1000
Checkpoint: 1250
Checkpoint: 1500
Checkpoint: 1750
Checkpoint: 2000
Checkpoint: 2250
Checkpoint: 2500
Checkpoint: 2750
Checkpoint: 3000
Checkpoint: 3250
Checkpoint: 3500
Checkpoint: 3750
Checkpoint: 4000
Checkpoint: 4250
Checkpoint: 4500
Checkpoint: 4750
Checkpoint: 5000
Checkpoint: 5250
Checkpoint: 5500
Checkpoint: 5750
Checkpoint: 6000
Checkpoint: 6250
Checkpoint: 6500
Checkpoint: 6750
Checkpoint: 7000
Checkpoint: 7250
Checkpoint: 7500
Checkpoint: 7750
Checkpoint: 8000
Checkpoint: 8250
Checkpoint: 8500
Checkpoint: 8750
Checkpoint: 9000
Checkpoint: 9250
Checkpoint: 9500
Checkpoint: 9750
Checkpoint: 10000
Checkpoint: 10250
Checkpoint: 10500
Checkpoint: 10750


In [ ]:
np.set_printoptions(precision=6)
jsonBatch = []
for f_ in lowScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/lowScore.p','wb'))
print('33 % Complete')
#################################################################
jsonBatch = []
for f_ in midScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/midScore.p','wb'))
print('67 % Complete')
#################################################################
jsonBatch = []
for f_ in highScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
print(pprint.pprint(jsonBatch))
pickle.dump(jsonBatch, open('.temp/highScore.p','wb'))
print('100 % Complete')

In [53]:
t1 = ('super', -23)
t2 = ('super', -23)
t1 == t2

True